In this competetition, our goal is to predict pawpularity of pet's profile based on appearance of pet profile. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Loading the .csv file

In [ ]:
tran=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
tran.head()

Listing train columns name

In [ ]:
tran.columns

Listing test columns names

In [ ]:
test.columns

Filtering columns for x and y variables

In [ ]:
column=[col for col in tran.columns if col not in ['Id','Pawpularity']]

Spliting train and test dataset

In [ ]:
x=tran[column]
y=tran['Pawpularity']

In [ ]:
print(x.shape,y.shape,test.shape)

In [ ]:
test_id=test['Id']
test=test.drop('Id',axis=1)
test.shape

Checking any null instances in train and test dataset

In [ ]:
tran.isnull().any()

In [ ]:
test.isnull().any()

Droping train id column

In [ ]:
tran.drop('Id',axis=1,inplace=True)

**Heatmap analysis**

In [ ]:
tran_matrix=tran.corr()
plt.figure(figsize=(10,8))
sns.heatmap(tran_matrix,linewidth=0.1,linecolor='white',annot=True,cmap='Pastel1',fmt='.2f')
plt.show()

In [ ]:
feature_variables=x.columns
target_variable='Pawpularity'

sns.countplot(x=feature_variables,y=target_variable,data=tran)

Using PCA to reduce dimension for analysis outlier

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2,random_state=42)
pca=pca.fit_transform(x)

Visualization of PCA 

In [ ]:
plt.figure(figsize=(8,7))
plt.scatter(pca[:,0],pca[:,1],color='red')
plt.show()

Checking pawpularity distribution 

In [ ]:
plt.hist(y,bins=10)
plt.show()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
import tensorflow as tf

In [ ]:
mm=MinMaxScaler()

In [ ]:
y_ss=y.values.reshape((-1,1))

In [ ]:
y_mm=mm.fit_transform(y_ss)

In [ ]:
print(y_mm.mean(),y_mm.max(),y_mm.min())

In [ ]:
pd.DataFrame(y_mm).head()

In [ ]:
print(x.shape,y_mm.shape)

In [ ]:
x_mm=x.copy()

Reshaping x and y dataset for neural network training purposes

In [ ]:
x_mm=np.array(x_mm).reshape(-1,8,x_mm.shape[1])
y_mm=np.array(y_mm).reshape(-1,8)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x_mm,y_mm,random_state=42,test_size=0.2)

In [ ]:
print(xtrain.shape,ytrain.shape)

**Model of neural network**

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(8,12)),
    tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dense(64,activation='relu'),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dense(1)
])

model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             metrics=['mse'])
hst=model.fit(xtrain,ytrain,
             epochs=100,validation_data=(xtest,ytest))

In [ ]:
hst.history.keys()

**Ploting training and validation losses**

In [ ]:
plt.plot(hst.history['loss'],label='Train_loss')
plt.plot(hst.history['val_loss'],label='Val_loss')
plt.legend()
plt.show()

**Visualizing training and validation mse**

In [ ]:
plt.plot(hst.history['mse'],label='Train_mse')
plt.plot(hst.history['val_mse'],label='Val_mse')
plt.legend()
plt.show()

In [ ]:
np.mean(hst.history['mse'])

Reshaping test dataset for predicting purposes

In [ ]:
test=np.array(test).reshape(-1,8,test.shape[1])

In [ ]:
sub=model.predict(test)

In [ ]:
sub=sub.reshape(-1,1)

In [ ]:
sub.shape

In [ ]:
sub=mm.inverse_transform(sub)

In [ ]:
submission=pd.DataFrame(sub,columns=['Pawpularity'])
submission=pd.concat([test_id,submission],axis=1)


In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)